In [0]:
%run ../dao/coingeckoapi_dao/

In [0]:
%run ../utils/save_helper/

In [0]:
from pyspark.sql.types import *

In [0]:
cgk_api = CoinGeckoAPI_DAO()

In [0]:
top_15_coins_market_cap = spark.sql("""SELECT id FROM api_coingecko.bronze_market_data_coins WHERE market_cap_rank <= 15""").collect()

list_top_15 = [row['id'] for row in top_15_coins_market_cap]

In [0]:
schema = StructType([
    StructField("coin_id", StringType(), True),
    StructField("vs_currency", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("market_cap", DoubleType(), True),
    StructField("volume_24h", DoubleType(), True),
    StructField("change_24h", DoubleType(), True),
    StructField("last_updated_at", LongType(), True)
])

In [0]:
if list_top_15:
    try:
        all_coins = ','.join(list_top_15)
        
        print(f"Buscando preços para: {all_coins.split(',')}")
        
        response_all = cgk_api.make_request('simple/price', params={
            'ids': all_coins,
            'vs_currencies': 'usd',
            'include_market_cap': 'true',
            'include_24hr_vol': 'true',
            'include_24hr_change': 'true',
            'include_last_updated_at': 'true'
        })
        
        all_data = []
        
        if response_all:
            for coin_id, coin_data in response_all.items():
                row_data = {
                    'coin_id': coin_id,
                    'vs_currency': 'usd',
                    'price': coin_data.get('usd', None),
                    'market_cap': coin_data.get('usd_market_cap', None),
                    'volume_24h': coin_data.get('usd_24h_vol', None),
                    'change_24h': coin_data.get('usd_24h_change', None),
                    'last_updated_at': coin_data.get('last_updated_at', None)
                }
                all_data.append(row_data)
                print(f"✓ {coin_id}: ${coin_data.get('usd', 'N/A')}")
            
            df_top_15 = spark.createDataFrame(all_data, schema)
            
            print(f"\nDataFrame TOP 15 - Total: {df_top_15.count()} registros")
            df_top_15.show(truncate=False)
            
        else:
            print("Nenhum dado retornado pela API")
            
    except Exception as e:
        print(f"Erro na busca do TOP 15: {e}")

In [0]:
def insert_with_merge(df_new_data):
    """Usa MERGE para fazer upsert dos dados"""
    
    print("Executando merge/upsert...")
    
    df_new_data.createOrReplaceTempView("temp_new_data")
    
    spark.sql("""
        MERGE INTO workspace.api_coingecko.bronze_top_15_price AS target
        USING temp_new_data AS source
        ON target.coin_id = source.coin_id 
        AND target.last_updated_at = source.last_updated_at
        
        WHEN NOT MATCHED THEN
        INSERT (coin_id, vs_currency, price, market_cap, volume_24h, change_24h, last_updated_at)
        VALUES (source.coin_id, source.vs_currency, source.price, source.market_cap, 
                source.volume_24h, source.change_24h, source.last_updated_at)
    """)
    
    print("✓ Merge concluído")

In [0]:
insert_with_merge(df_top_15)

In [0]:
%sql
SELECT * FROM workspace.api_coingecko.bronze_top_15_price